### Checking the model inference

In [1]:
import sys
sys.path.append('..')

import numpy as np
import sqlite3 as sq
from src.model import USEQA
    
# string addresses and questions
query_string = "Is Genesis a scientific document?"
vectorised_master_kb_dir = '../data/vectorised_master_sentence_kb.npz'
master_kb_text_dir = '../data/master_kb_sentence_text_clean.db'

# load master kb
with np.load(vectorised_master_kb_dir) as data:
    master_matrix = data['arr_0']
    
# sql connection
con = sq.connect(master_kb_text_dir)

In [3]:
# init model
model = USEQA()

In [18]:
data_df, top_scores = model.make_query(query_string, master_matrix, con)
display(data_df)
display(top_scores)

,sentence,paragraph,authors,document_nm,url,verse_references
idx,,,,,,
186,"Intimate communion with God, a spirit trained ...","Accepting this chapter then as it stands, and ...",Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-3
1141,This is seen to be an end worthy of all that i...,Here the beginning and the end of God’s revela...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-2
39,Of the existence of fruit trees prior to the e...,Now many persons cut the knot by simply denyin...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-2
6977,"Apart from Him it had no meaning, and would ha...",JACOB’S blessing of his sons marks the close o...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-50
8557,V,Nor would the names of the children of the fou...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/e...,Exodus-7


array([0.10903956, 0.04264995, 0.12735684, 0.15464602, 0.04423378],
      dtype=float32)

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
k=0
cosine_similarity(
    model.predict(data_df.sentence.iloc[k],data_df.paragraph.iloc[k], type='response'),
    model.predict(query_string, type='query')
)

array([[0.28925547]], dtype=float32)

In [57]:
encoded_res = model.predict(sentence, paragraph, type='response')
encoded_que = model.predict(query_string, type='query')

similarity_score= cosine_similarity(master_matrix, encoded_que)
display(sims[:4])
display(f"Max: {sims.max()}")

array([[0.36601952],
       [0.30955666],
       [0.4297026 ],
       [0.2522238 ]], dtype=float32)

'Max: 0.5348503589630127'

In [54]:
sims.argmax()

52218

In [98]:
from src.utils import query_SQL_by_idx

query_SQL_by_idx(con, 59094)

[(59094.0,
  'The first chapter of Genesis is the classic example of the appropriation by an inspired writer of the scientific spirit and method',
  'Chronicles is not the most important witness to a sympathetic relationship between the Bible and exact science. The first chapter of Genesis is the classic example of the appropriation by an inspired writer of the scientific spirit and method. Some chapters in Job show a distinctly scientific interest in natural phenomena. Moreover, the direct concern of Chronicles is in the religious aspects of social science. And yet there is a patient accumulation of data with no obvious dramatic value: names, dates, numbers, specifications, and ritual which do not improve the literary character of the narrative. This conscientious recording of dry facts, this noting down of anything and everything that connects with the subject, is closely akin to the initial processes of the inductive sciences. True, the chronicler’s interests are in some directions 

### The current `make_query()` function
```
def make_query(self, querystring, vectorized_knowledge, con, k=5, predict_type='query'):
    """Make a query against the stored vectorized knowledge. 
    Choose index=True to return sorted index of matches.
    type can be 'query' or 'response' if you are comparing statements
    """
    # score the stuff
    similarity_score=cosine_similarity(vectorized_knowledge, self.predict([querystring], type=predict_type))
    top_k_scores = similarity_score[similarity_score.argsort(axis=0)<k]
    sortargs=np.flip(similarity_score.argsort(axis=0))
    sortargs = [sortarg[0] for sortarg in sortargs]

    # indices of best k answers
    idx_of_best_k_answers = find_idx_of_best_k_answers(sortargs, k)

    # call from sql
    #list_of_context_tuples = [query_SQL_by_idx(con, idx) for idx in idx_of_best_k_answers]
    data_df = pds.read_sql('SELECT * FROM master_kb_text WHERE idx IN (' + str(idx_of_best_k_answers)[1:-1] + ')', con)
    data_df = data_df.set_index('idx').loc[idx_of_best_k_answers]
    return data_df, top_k_scores
        
```

In [107]:
idx_of_best_k_answers = np.argsort(similarity_score.flatten().tolist())[-5:]
idx_of_best_k_answers

array([60897, 58758, 52218, 59094, 55488])

Arg sort is the better sorter.  
`>>>np.argsort([5,4,6,6,8,1])`  
`>>>array([5, 1, 0, 2, 3, 4])`  
Intuitively, (1) argsort gives us the indices of increasingly higher values (2) by indexing [-5:] we are picking out the top 5 indices

### Perfectly correct encoding

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

query_string = "Is Genesis a scientific document?"
sentence = "IF anyone is in search of accurate information regarding the age of this earth, or its relation to the sun, moon, and stars, or regarding the order in which plants and animals have appeared upon it, he is referred to recent textbooks in astronomy, geology, and palaeontology."
paragraph = "IF anyone is in search of accurate information regarding the age of this earth, or its relation to the sun, moon, and stars, or regarding the order in which plants and animals have appeared upon it, he is referred to recent textbooks in astronomy, geology, and palaeontology. No one for a moment dreams of referring a serious student of these subjects to the Bible as a source of information. It is not the object of the writers of Scripture to impart physical instruction or to enlarge the bounds of scientific knowledge. But if any one wishes to know what connection the world has with God, if he seeks to trace back all that now is to the very fountain-head of life, if he desires to discover some unifying principle, some illuminating purpose in the history of this earth, then we confidently refer him to these and the subsequent chapters of Scripture as his safest, and indeed his only, guide to the information he seeks. Every writing must be judged by the object the writer has in view. If the object of the writer of these chapters was to convey physical information, then certainly it is imperfectly fulfilled. But if his object was to give an intelligible account of God’s relation to the world and to man, then it must be owned that he has been successful in the highest degree."

cosine_similarity(
    model.predict(sentence, paragraph, type='response'),
    model.predict(query_string, type='query')
)
print(cosine_sim)

data_df, top_scores = model.make_query(query_string, master_matrix, con)
display(data_df)
display(top_scores)

array([[0.36601955]], dtype=float32)

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

query_string = "Was the book of Philemon really written by Apostle Paul or was it forged?"
sentence = "The book claims that the Apostle Paul was its writer (vv. 1, 9, 19), a claim that few in the history of the church have disputed, especially since there is nothing in Philemon that a forger would have been motivated to write."
paragraph = "The book claims that the Apostle Paul was its writer (vv. 1, 9, 19), a claim that few in the history of the church have disputed, especially since there is nothing in Philemon that a forger would have been motivated to write. It is one of the Prison Epistles, along with Ephesians, Philippians, and Colossians. Its close connection with Colossians, which Paul wrote at the same time (ca. A.D. 60–62; cf. vv. 1, 16), brought early and unquestioned vindication of Paul’s authorship by the early church fathers (e.g., Jerome, Chrysostom, and Theodore of Mopsuestia). The earliest of NT canons, the Muratorian (ca. A.D. 170), includes Philemon. For biographical information on Paul, see Introduction to Romans: Author and Date; for the date and place of Philemon’s writing, see Introductions to Ephesians and Philippians: Author and Date."

cosine_sim = cosine_similarity(
    model.predict(sentence, paragraph, type='response'),
    model.predict(query_string, type='query')
)
print(cosine_sim)

data_df, top_scores = model.make_query(query_string, master_matrix, con)
display(data_df)
display(top_scores)

[[0.6785306]]


,sentence,paragraph,authors,document_nm,url,verse_references
idx,,,,,,
137977,It may have been composed almost contemporaneo...,This letter was written from prison in Rome (A...,Macarthur John,- Bible Introductions - Colossians,https://www.blueletterbible.org/Comm/macarthur...,
137893,Another alternative has been that Paul wrote t...,Another alternative has been that Paul wrote t...,Macarthur John,- Bible Introductions - Philippians,https://www.blueletterbible.org/Comm/macarthur...,
138183,The book claims that the Apostle Paul was its ...,The book claims that the Apostle Paul was its ...,Macarthur John,- Bible Introductions - Philemon,https://www.blueletterbible.org/Comm/macarthur...,
137839,It may have been composed almost contemporaneo...,There is no indication that the authorship of ...,Macarthur John,- Bible Introductions - Ephesians,https://www.blueletterbible.org/Comm/macarthur...,
137945,Additional evidence for Paul’s authorship come...,Paul is identified as author at the beginning ...,Macarthur John,- Bible Introductions - Colossians,https://www.blueletterbible.org/Comm/macarthur...,


array([[0.57329506],
       [0.5774895 ],
       [0.5821841 ],
       [0.58623827],
       [0.6483907 ]], dtype=float32)

### A better sentence split
`split('.')` breaks abbrevs too which is not what we want. A better split would be to use spacy's sentence split instead. 


In [11]:
from nltk import tokenize
def sentence_tokenize(paragraph):
    """
    https://stackoverflow.com/questions/4576077/python-split-text-on-sentences
    """
    return tokenize.sent_tokenize(paragraph)

sentence_tokenize(sentence)

['The book claims that the Apostle Paul was its writer (vv.',
 '1, 9, 19), a claim that few in the history of the church have disputed, especially since there is nothing in Philemon that a forger would have been motivated to write.']

In [19]:
import spacy
nlp = spacy.load('en')

def sentence_tokenize(paragraph):
    """
    https://stackoverflow.com/questions/4576077/python-split-text-on-sentences
    """
    tokens = nlp(paragraph)
    return [str(sent.string.strip()) for sent in tokens.sents]

sentence_tokenize(paragraph)

['The book claims that the Apostle Paul was its writer (vv. 1, 9, 19), a claim that few in the history of the church have disputed, especially since there is nothing in Philemon that a forger would have been motivated to write.',
 'It is one of the Prison Epistles, along with Ephesians, Philippians, and Colossians.',
 'Its close connection with Colossians, which Paul wrote at the same time (ca.',
 'A.D. 60–62;',
 'cf.',
 'vv. 1, 16), brought early and unquestioned vindication of Paul',
 '’s authorship by the early church fathers (e.g., Jerome, Chrysostom, and Theodore of Mopsuestia).',
 'The earliest of NT canons, the Muratorian (ca. A.D. 170), includes Philemon.',
 'For biographical information on Paul, see Introduction to Romans:',
 'Author and Date; for the date and place of Philemon’s writing, see Introductions to Ephesians and Philippians: Author and Date.']

### Checking `make_query()` function.

In [1]:
import sys
sys.path.append('..')

import numpy as np
import sqlite3 as sq
from src.model import USEQA
    
# string addresses and questions
query_string = "Is Genesis a scientific document?"
vectorised_master_kb_dir = '../data/vectorised_master_sentence_kb.npz'
master_kb_text_dir = '../data/master_kb_sentence_text.db'

# load master kb
with np.load(vectorised_master_kb_dir) as data:
    master_matrix = data['arr_0']
    
# sql connection
con = sq.connect(master_kb_text_dir)
    
model = USEQA()
data_df, top_scores = model.make_query(query_string, master_matrix, con)
display(data_df)
display(top_scores)

model initiated!


,sentence,paragraph,authors,document_nm,url,verse_references
idx,,,,,,
60897,The first chapter of Genesis is the classic ex...,Chronicles is not the most important witness t...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/1...,1_chronicles-6
58758,The first chapter of Genesis is the classic ex...,Chronicles is not the most important witness t...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/1...,1_chronicles-4
52218,The first chapter of Genesis is the classic ex...,Chronicles is not the most important witness t...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/1...,1_chronicles-2
55488,The first chapter of Genesis is the classic ex...,Chronicles is not the most important witness t...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/1...,1_chronicles-3
59094,The first chapter of Genesis is the classic ex...,Chronicles is not the most important witness t...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/1...,1_chronicles-5


array([[0.5348503 ],
       [0.5348503 ],
       [0.53485036],
       [0.53485036],
       [0.53485036]], dtype=float32)

### Appendix: Correct cleaning and handling of indices

In [3]:
master_kb_text_dir = '../data/master_kb_sentence_text.db'
    
# sql connection
con = sq.connect(master_kb_text_dir)

import pandas.io.sql as pds
data_df = pds.read_sql('SELECT * FROM master_kb_text', con)

data_df = data_df.set_index(np.arange(len(data_df)))
data_df.head(3)

,idx,sentence,paragraph,authors,document_nm,url,verse_references
0,0,IF anyone is in search of accurate information...,IF anyone is in search of accurate information...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-2
1,1,No one for a moment dreams of referring a seri...,IF anyone is in search of accurate information...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-2
2,2,It is not the object of the writers of Scriptu...,IF anyone is in search of accurate information...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-2


In [4]:
# dropping duplicates
print(f"{len(data_df)} before")
data_df = data_df.drop_duplicates('sentence')
print(f"{len(data_df)} after")

147003 before
147003 after


The unclean base has idx correctly, but it has an index there

In [5]:
data_df = data_df.assign(idx = np.arange(len(data_df)))
data_df = data_df.set_index('idx')

In [6]:
import os
master_kb_text_clean_dir="../data/master_kb_sentence_text_clean.db"

if os.path.exists(master_kb_text_clean_dir):
    os.remove(master_kb_text_clean_dir)
con = sq.connect(master_kb_text_clean_dir)
query = 'CREATE TABLE master_kb_text (idx real, sentence text, paragraph text, authors text, document_nm text, url text, verse_references text)'
con.execute(query)
data_df.to_sql('master_kb_text', con=con, if_exists='replace')
# con.close()

In [3]:
import pandas.io.sql as pds
data_df = pds.read_sql('SELECT * FROM master_kb_text WHERE idx IN (' + str(list([1,2]))[1:-1] + ')', con)
data_df

,idx,sentence,paragraph,authors,document_nm,url,verse_references
0,1,No one for a moment dreams of referring a seri...,IF anyone is in search of accurate information...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-2
1,2,It is not the object of the writers of Scriptu...,IF anyone is in search of accurate information...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-2


In [7]:
data_df = data_df[data_df.sentence.apply(len) > 4]

,idx,sentence,paragraph,authors,document_nm,url,verse_references
0,1,No one for a moment dreams of referring a seri...,IF anyone is in search of accurate information...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-2
1,2,It is not the object of the writers of Scriptu...,IF anyone is in search of accurate information...,Expositors Commentary,,https://biblehub.com/commentaries/expositors/g...,Genesis-2


In [8]:
con.close()